In [27]:
!which python

/home/user/jfayzullaev/stellar-clustering/.venv-vis/bin/python


In [28]:
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import normalized_mutual_info_score as NMI, adjusted_rand_score as ARI


In [29]:
KMEANS_FILE_TX   = "transactions/transactions_role2vec_kmeans_results.csv"
KMEANS_FILE_TL   = "trustlines/trustlines_role2vec_kmeans_results.csv"

In [30]:
NORM_LABELS_PATH = "~/stellar-clustering/network/labled-data/labels/label-normalization/labels_entities_normalized.csv"  

In [31]:
N_SPLITS     = 5
RANDOM_STATE = 42

In [ ]:
def overall_purity_comm(df_comm_name: pd.DataFrame, comm_col: str, name_col: str = "name") -> float:

    if df_comm_name.empty:
        return np.nan
    counts = df_comm_name.groupby([comm_col, name_col]).size().reset_index(name='cnt')
    totals = counts.groupby(comm_col)['cnt'].sum()
    max_per_comm = counts.groupby(comm_col)['cnt'].max()
    return float(max_per_comm.sum() / totals.sum())


In [ ]:
def load_kmeans_fixed_k(kmeans_file: str, k_col: str) -> pd.DataFrame:

    df = pd.read_csv(kmeans_file)

    df = df[['account_id', k_col]].dropna().drop_duplicates()
    df = df.rename(columns={k_col: 'cluster'})


    try:
        df['account_id'] = df['account_id'].astype(int)
    except Exception:
        df['account_id'] = df['account_id'].astype(str)
    return df

In [ ]:
def evaluate_fixed_kmeans_cv(
    labels_path: str,
    clusters_df: pd.DataFrame,
    label_col: str = "name",
    n_splits: int = 5,
    random_state: int = 42
):

    if not os.path.exists(labels_path):
        raise FileNotFoundError(f"Labels file not found: {labels_path}")

    labels = (pd.read_csv(labels_path)
                .dropna(subset=['account_id', label_col])
                .drop_duplicates(subset=['account_id'])
                .rename(columns={label_col: 'name'}))

    try:
        labels['account_id'] = labels['account_id'].astype(int)
        clu = clusters_df.copy()
        clu['account_id'] = clu['account_id'].astype(int)
    except Exception:
        labels['account_id'] = labels['account_id'].astype(str)
        clu = clusters_df.copy()
        clu['account_id'] = clu['account_id'].astype(str)

    joined = labels.merge(clu, on='account_id', how='inner')

    n_labeled = len(labels)
    n_joined  = len(joined)
    coverage  = (n_joined / n_labeled) if n_labeled else 0.0
    if n_joined == 0:
        raise ValueError("No labeled accounts found in this K-Means file after join.")

    le = LabelEncoder()
    y_all = le.fit_transform(joined['name'].values)
    X_ids = joined['account_id'].values

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    rows = []

    for fold, (tr_idx, te_idx) in enumerate(skf.split(X_ids, y_all), start=1):
        df_tr = joined.iloc[tr_idx].copy()
        df_te = joined.iloc[te_idx].copy()

        y_true_tr = le.transform(df_tr['name'])
        y_pred_tr = df_tr['cluster'].values
        nmi_tr = NMI(y_true_tr, y_pred_tr)
        ari_tr = ARI(y_true_tr, y_pred_tr)
        purity_tr = overall_purity_comm(df_tr[['cluster', 'name']], comm_col='cluster', name_col='name')

        y_true_te = le.transform(df_te['name'])
        y_pred_te = df_te['cluster'].values
        nmi_te = NMI(y_true_te, y_pred_te)
        ari_te = ARI(y_true_te, y_pred_te)
        purity_te = overall_purity_comm(df_te[['cluster', 'name']], comm_col='cluster', name_col='name')

        rows.append({
            'fold': fold,
            'n_train': len(df_tr),
            'n_test': len(df_te),
            'train_frac': len(df_tr) / len(joined),
            'NMI_train': nmi_tr,
            'ARI_train': ari_tr,
            'Purity_train': purity_tr,
            'NMI_test': nmi_te,
            'ARI_test': ari_te,
            'Purity_test': purity_te,
        })

    per_fold_df = pd.DataFrame(rows)

    averages = {
        'Avg_NMI_train': float(np.nanmean(per_fold_df['NMI_train'].values)),
        'Avg_ARI_train': float(np.nanmean(per_fold_df['ARI_train'].values)),
        'Avg_Purity_train': float(np.nanmean(per_fold_df['Purity_train'].values)),
        'Avg_NMI_test': float(np.nanmean(per_fold_df['NMI_test'].values)),
        'Avg_ARI_test': float(np.nanmean(per_fold_df['ARI_test'].values)),
        'Avg_Purity_test': float(np.nanmean(per_fold_df['Purity_test'].values)),
        'Avg_train_frac': float(np.nanmean(per_fold_df['train_frac'].values)),
    }

    coverage_info = {
        'n_labeled': int(n_labeled),
        'n_joined': int(n_joined),
        'coverage': float(coverage),
    }

    return per_fold_df, averages, coverage_info



## Transactions

In [35]:
K_COL = "kmeans_15" 

In [ ]:
clusters_df = load_kmeans_fixed_k(KMEANS_FILE_TX, K_COL)

In [37]:
norm_labels_path = os.path.expanduser(NORM_LABELS_PATH)

In [38]:
norm_per_fold, norm_avg, norm_cov = evaluate_fixed_kmeans_cv(
    labels_path=norm_labels_path,
    clusters_df=clusters_df,
    label_col="name",
    n_splits=N_SPLITS,
    random_state=RANDOM_STATE
)

/home/user/jfayzullaev/stellar-clustering/.venv-vis/lib/python3.9/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [39]:
print(f"\n K-Means {K_COL} | Normalized Labels per fold metrics")
display(norm_per_fold)


 K-Means kmeans_15 | Normalized Labels per fold metrics


,fold,n_train,n_test,train_frac,NMI_train,ARI_train,Purity_train,NMI_test,ARI_test,Purity_test
0,1,317,80,0.798489,0.303689,0.057977,0.747634,0.361473,0.078462,0.750000
1,2,317,80,0.798489,0.312242,0.072994,0.750789,0.365918,0.026112,0.762500
2,3,318,79,0.801008,0.314169,0.068695,0.751572,0.318493,0.026778,0.734177
3,4,318,79,0.801008,0.313317,0.072796,0.742138,0.325253,0.031447,0.759494
4,5,318,79,0.801008,0.289091,0.043811,0.742138,0.409059,0.153521,0.759494


In [40]:
print(f"\n K-Means {K_COL} | Normalized Labels avgs")
for k, v in norm_avg.items():
    print(f"{k}: {v:.6f}")
print(f"Coverage: {norm_cov['coverage']:.2%}  ({norm_cov['n_joined']}/{norm_cov['n_labeled']})")



 K-Means kmeans_15 | Normalized Labels avgs
Avg_NMI_train: 0.306502
Avg_ARI_train: 0.063254
Avg_Purity_train: 0.746854
Avg_NMI_test: 0.356039
Avg_ARI_test: 0.063264
Avg_Purity_test: 0.753133
Avg_train_frac: 0.800000
Coverage: 4.90%  (397/8102)


In [41]:
TX_PATH = 'transactions/cross-validation'

In [42]:
os.makedirs(TX_PATH, exist_ok=True)

In [43]:
norm_per_fold.to_csv(f"{TX_PATH}/tx_cv_{K_COL}_norm_per_fold.csv", index=False)
pd.DataFrame([{
    **norm_avg,
    **norm_cov,
    "k_col": K_COL,
    "source_file": os.path.basename(KMEANS_FILE_TX)
}]).to_csv(f"{TX_PATH}/tx_cv_{K_COL}_norm_summary.csv", index=False)
print(f"Saved to {TX_PATH}")

Saved to transactions/cross-validation


## Trustlines

In [44]:
K_COL = "kmeans_30"

In [ ]:
clusters_df = load_kmeans_fixed_k(KMEANS_FILE_TL, K_COL)

In [46]:
norm_labels_path = os.path.expanduser(NORM_LABELS_PATH)

In [47]:
norm_per_fold, norm_avg, norm_cov = evaluate_fixed_kmeans_cv(
    labels_path=norm_labels_path,
    clusters_df=clusters_df,
    label_col="name",
    n_splits=N_SPLITS,
    random_state=RANDOM_STATE
)

/home/user/jfayzullaev/stellar-clustering/.venv-vis/lib/python3.9/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [48]:
print(f"\n K-Means {K_COL} | Normalized Labels per fold metrics")
display(norm_per_fold)


 K-Means kmeans_30 | Normalized Labels per fold metrics


,fold,n_train,n_test,train_frac,NMI_train,ARI_train,Purity_train,NMI_test,ARI_test,Purity_test
0,1,111,28,0.798561,0.243137,0.020392,0.891892,0.295502,0.007045,0.928571
1,2,111,28,0.798561,0.257865,0.016583,0.900901,0.349589,0.063859,0.928571
2,3,111,28,0.798561,0.274096,0.022208,0.900901,0.303823,0.015917,0.928571
3,4,111,28,0.798561,0.256316,0.021717,0.900901,0.344774,0.024699,0.964286
4,5,112,27,0.805755,0.257512,0.021605,0.875000,0.204908,0.012992,1.000000


In [49]:
print(f"\n K-Means {K_COL} | Normalized Labels avgs")
for k, v in norm_avg.items():
    print(f"{k}: {v:.6f}")
print(f"Coverage: {norm_cov['coverage']:.2%}  ({norm_cov['n_joined']}/{norm_cov['n_labeled']})")



 K-Means kmeans_30 | Normalized Labels avgs
Avg_NMI_train: 0.257785
Avg_ARI_train: 0.020501
Avg_Purity_train: 0.893919
Avg_NMI_test: 0.299719
Avg_ARI_test: 0.024902
Avg_Purity_test: 0.950000
Avg_train_frac: 0.800000
Coverage: 1.72%  (139/8102)


In [50]:
TL_PATH = 'trustlines/cross-validation'

In [51]:
os.makedirs(TL_PATH, exist_ok=True)

In [52]:
norm_per_fold.to_csv(f"{TL_PATH}/tx_cv_{K_COL}_norm_per_fold.csv", index=False)

pd.DataFrame([{
    **norm_avg,
    **norm_cov,
    "k_col": K_COL,
    "source_file": os.path.basename(KMEANS_FILE_TL)
}]).to_csv(f"{TL_PATH}/tx_cv_{K_COL}_norm_summary.csv", index=False)
print(f"Saved to {TL_PATH}")

Saved to trustlines/cross-validation
